## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bradesti,RO,44.4833,23.6333,29.89,77,79,11.77,broken clouds
1,1,Vangaindrano,MG,-23.3500,47.6000,73.04,85,0,5.06,clear sky
2,2,Albany,US,42.6001,-73.9662,8.85,61,100,15.99,overcast clouds
3,3,Half Moon Bay,US,37.4636,-122.4286,64.08,61,0,1.99,clear sky
4,4,Esperance,AU,-33.8667,121.9000,76.84,49,2,9.01,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                      (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vangaindrano,MG,-23.3500,47.6000,73.04,85,0,5.06,clear sky
4,4,Esperance,AU,-33.8667,121.9000,76.84,49,2,9.01,clear sky
13,13,Butaritari,KI,3.0707,172.7902,82.56,72,78,18.01,broken clouds
22,22,Bowen,AU,-20.0167,148.2333,82.62,56,3,12.84,clear sky
27,27,New Norfolk,AU,-42.7826,147.0587,75.65,31,0,2.93,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vangaindrano,MG,73.04,clear sky,-23.3500,47.6000,
4,Esperance,AU,76.84,clear sky,-33.8667,121.9000,
13,Butaritari,KI,82.56,broken clouds,3.0707,172.7902,
22,Bowen,AU,82.62,clear sky,-20.0167,148.2333,
27,New Norfolk,AU,75.65,clear sky,-42.7826,147.0587,
32,Avarua,CK,84.25,few clouds,-21.2078,-159.7750,
36,Hobart,AU,77.02,few clouds,-42.8794,147.3294,
37,Bima,ID,82.80,overcast clouds,-8.4667,118.7167,
39,Sibut,CF,71.65,few clouds,5.7180,19.0739,
41,Ixtapa,MX,75.22,clear sky,20.7000,-105.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f"{lat},{lng}"
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except:
        print('Hotel not found...skipping.')
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vangaindrano,MG,73.04,clear sky,-23.3500,47.6000,Hotel Fanilo
4,Esperance,AU,76.84,clear sky,-33.8667,121.9000,Hospitality Esperance
13,Butaritari,KI,82.56,broken clouds,3.0707,172.7902,Isles Sunset Lodge
22,Bowen,AU,82.62,clear sky,-20.0167,148.2333,Castle Motor Lodge Motel
27,New Norfolk,AU,75.65,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages
32,Avarua,CK,84.25,few clouds,-21.2078,-159.7750,Paradise Inn
36,Hobart,AU,77.02,few clouds,-42.8794,147.3294,Mantra on Collins Hobart
37,Bima,ID,82.80,overcast clouds,-8.4667,118.7167,Lambitu Hotel
41,Ixtapa,MX,75.22,clear sky,20.7000,-105.2000,Hotel Plaza Providencia
42,Kapaa,US,80.58,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))